## Prepare migest input based on parentage results generated in ~/parentage/notebooks/process_colony_dyad & and demography measures from~/parentage/notebooks/proportion_sampled_allison

In [21]:
Packages <- c("dplyr", "sjmisc", "RMySQL",  "tidyr","tidyverse", "lubridate")


invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/migest')

source("~/scripts/conleyte.R")
source("~/scripts/conlabor.R")
source("~/scripts/sample_latlon.R")
source("~/scripts/get_date.R")
source("~/scripts/get_samp.R")
source("~/scripts/anem_meta.R")
source("~/scripts/get_fish.R")
source("~/scripts/get_lig.R")
source("~/scripts/create_text.R")
load("~/parentage/r_data/sampled_area_each_year.RData")
load("~/parentage/r_data/anems_visited_by_year.RData")
load("~/parentage/r_data/total_sampling_across_years.RData")
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")


labor <- conlabor()
leyte <- conleyte()



# download the fish_Obsfile
download.file(url = "https://github.com/pinskylab/genomics/blob/master/data/fish-obs.RData?raw=true", destfile = "~/parentage/r_data/fish-obs.RData")

# read in the data
fish_obs <- readRDS("~/parentage/r_data/fish-obs.RData") 



In [22]:
#read in parentage matches
mums <- read.csv(file="~/parentage/colony2/20190523_1340loci/results/1340_colony_migest_mums_allyears.csv", header=TRUE)
dads <- read.csv(file="~/parentage/colony2/20190523_1340loci/results/1340_colony_migest_dads_allyears.csv", header=TRUE)
N_trios <- read.csv(file="~/parentage/colony2/20190523_1340loci/results/1340_colony_migest_trios_allyears.csv", header=TRUE)

#read in demography estimates
prop_samp <- cumulative_prop_hab_sampled_by_site %>%
    mutate(total_possible_sample_anems = ifelse(site=="Caridad Proper", 4, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Caridad Proper" & total_anems_sampled==4, 1, total_prop_hab_sampled_anems_tidied) ) %>%
    mutate(total_possible_sample_anems = ifelse(site=="Sitio Lonas", total_anems_sampled, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Sitio Lonas", 1, total_prop_hab_sampled_anems_tidied) ) %>%
    select(site, end_year, total_prop_hab_sampled_anems_tidied) %>%
    rename(prop_habitat_sampled = "total_prop_hab_sampled_anems_tidied", year="end_year") 


prop_samp$prop_habitat_sampled[is.nan(prop_samp$prop_habitat_sampled)] <- 0
#prop_samp$site <- gsub(". ", ".", prop_samp$site, fixed=TRUE)

#prop_samp_annual <- sampled_area_each_year %>%
#    filter(method=="metal tags"& total_area_method=="anems tidied") %>%
#    select(time_frame, site, prop_hab_sampled_tidied)
#prop_samp_annual$site <- gsub(". ", ".", prop_samp_annual$site, fixed=TRUE)
#
#
#
##read in the sites which we sampled each year
N_gen_par <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_parents_corrected.txt", header = TRUE) %>% #not sure that I need the parents here
    select(year, sample_id, fish_indiv)

N_gen_offs <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_offspring_corrected.txt", header=T, stringsAsFactors = F) %>%
    select(year, sample_id, fish_indiv) 

total_gen <- bind_rows(N_gen_par, N_gen_offs)

meta <- get_fish() %>% #get all identifying info for a sample
    filter(sample_id %in% total_gen$sample_id | sample_id %in% total_gen$sample_id ) %>%
    distinct(sample_id, .keep_all = T) %>% 
    select(site, sample_id)
    
N_cap <- left_join(total_gen, meta, by="sample_id") %>%
    distinct(fish_indiv, .keep_all = T) %>%
    group_by(site, year) %>%
    summarise(n_cap=n())

N_cap_offs <- left_join(total_gen, meta, by="sample_id") %>%
    filter(sample_id %in% N_gen_offs$sample_id) %>%
    distinct(fish_indiv, .keep_all = T) %>%
    group_by(site, year) %>%
    summarise(n_cap=n())

sum(N_cap$n_cap) #should be 2396
sum(N_cap_offs$n_cap) #should be 785



#TEST for short distance dispersal patterns
#mums <- read.csv(file="~/migest/input/20180910colony_migest_testmums_shortdist.csv", header=TRUE)
#dads <- read.csv(file="~/migest/input/20180910colony_migest_testdads_shortdist.csv", header=TRUE)
#N_trios <- read.csv(file="~/migest/input/20180910colony_migest_testtrios_allyears.csv", header=TRUE)



##UPDATE NEXT
#read in demography estimates
#prop_samp <- read.csv(file="~/migest/input/20181017prop_habitat_samp.csv", header=TRUE)
#N_cap <- read.csv(file="~/migest/input/20181017_Nsampled.csv", header = TRUE)
#N_cap_offs <- read.csv(file="~/migest/input/20181017_Nsampled_offs.csv")

Warning message in bind_rows_(x, .id):
“binding factor and character vector, coercing into character vector”Warning message in bind_rows_(x, .id):
“binding character and factor vector, coercing into character vector”

[1] 2396

[1] 785

In [3]:
#iterate the input generation over all sites

In [4]:
#generate site list
prop_samp$site <- as.character(prop_samp$site)
prop_samp <- prop_samp %>% 
    arrange(site)%>%
    group_by(site) %>% #use these three bottom lines for average all years
    mutate(prop_habitat_sampled=mean(prop_habitat_sampled)) %>%
    distinct(site, .keep_all = TRUE) %>%
    select(-year) %>%
    ungroup()
#for annual
#prop_samp$year <- as.numeric(prop_samp$year)

prop_samp$prop_habitat_sampled <- round(prop_samp$prop_habitat_sampled, digits=4)


In [261]:
#there has to be a more elegant way to make an iput file per year, but for now, just manually filter N_cap, N_cap_offs, N_trios, and mums/dads to evaluate for in the for loop
mums <- mums %>%
    filter(year==2018)
dads <- dads %>%
    filter(year==2018)
N_trios <- N_trios %>%
    filter(year== 2018)
N_cap_offs <- N_cap_offs %>%
    filter(year==2018)
N_cap <- N_cap %>%
    filter(year==2018)
prop_samp <- prop_samp %>%    
    filter(year==2018) %>%
    ungroup() %>%
    select(site, prop_habitat_sampled)

In [8]:
sum(N_trios$n_trios)
sum(mums$n_mum)
sum(dads$n_dad)

[1] 24

[1] 20

[1] 18

In [5]:
#table of number of adults sampled

N_cap$year <- as.numeric(N_cap$year)
#N_cap2 <- N_cap %>%
#filter(year >= 2018 & year <= 2018) %>%
#group_by(site) %>%
#summarise(sampled_fish=sum(n_captured_fish))
    
N_cap2 <- N_cap %>%
    ungroup() %>%
    group_by(site) %>%
    mutate(n_male=(n_cap/2)) %>%
    mutate(n_female=(n_cap/2)) 
#######
#allyear 
cap_f <- N_cap2 %>% 
    group_by(site) %>%
    summarise(n_female=sum(n_female))
cap_f$n_female <- floor(cap_f$n_female)

cap_m <- N_cap2 %>% 
    group_by(site) %>%
    summarise(n_male=sum(n_male))
cap_m$n_male <- ceiling(cap_m$n_male)


N_cap2 <- left_join(cap_f, cap_m)

#######
#table of number of juveniles sampled
#N_cap_offs$year <- as.numeric(N_cap_offs$year)#careful with numeric vs character for year
N_cap_offs2 <- N_cap_offs %>%
    group_by(site) %>%
    summarise(sampled_fish=sum(n_cap))%>%
    ungroup()


N_cap_offs2$sampled_fish <- round(N_cap_offs2$sampled_fish, digits=0)



Joining, by = "site"


In [6]:
#use Ncap to generate sites list
sites1 <- N_cap_offs2 %>%
    ungroup() %>%
    select(site) %>%
    arrange(site)

sites2 <- N_cap2 %>%
    ungroup() %>%
    select(site) %>%
    arrange(site)
sites <- bind_rows(sites1, sites2)
sites <- sites %>%
    group_by(site) %>%
    filter(row_number()==1) %>%
    ungroup() %>%
    arrange(site)
sites$site <- as.character(sites$site)
#migest won't take site names, just pop numbers. The sites are in alphabetical order, here is their order with corresponding pop numbers
sites$pop <- c(seq(from=1, to=nrow(sites), by=1))




In [7]:
head(N_cap)

site,year,n_cap
Cabatoan,2012,17
Cabatoan,2013,11
Cabatoan,2014,7
Cabatoan,2015,32
Cabatoan,2016,28
Cabatoan,2018,9


In [8]:
#create empty data frame to add individual site tables to 
empt_dat2 <- as.data.frame(matrix(nrow=0, ncol=(nrow(sites))))
col <- c(sites$pop)
colnames(empt_dat2) <- col

#create an empty data frame to store site names
site_inc <- as.data.frame(matrix(nrow=0, ncol=1))
col <- "site"
colnames(site_inc) <- col

#check for correct parentage sum in these matrices at the end
sum_par <- as.data.frame(matrix(nrow=nrow(sites), ncol=1))
sum_trios <- as.data.frame(matrix(nrow=nrow(sites), ncol=1))




In [9]:
for(i in 1:length(sites$site)){
    site_eval <-  sites$site[i]
    #prepare the parentage matrix input
  
     if(site_eval %in% mums$offs_site){
    site_matF <- mums %>%
        filter(offs_site == site_eval) %>%
        select(offs_site, par_site, n_mum) %>%
        group_by(offs_site, par_site) %>%
        summarise(n_mum=(sum(n_mum))) }
    else {site_matF <- as.data.frame(matrix(nrow=1, ncol=3))
          site_matF[1,1] <- site_eval
          site_matF[1,2] <- site_eval
          site_matF[1,3] <- 0
          col <- c("offs_site", "par_site", "n_mum")
        colnames(site_matF) <- col
         }
    site_matF2 <- suppressWarnings(left_join(sites, site_matF, by=c(site ="par_site"))) 
        site_matF3 <- site_matF2 %>%
        select(n_mum)

    if(site_eval %in% dads$offs_site){
    site_matM <- dads %>%
        filter(offs_site == site_eval) %>%
        select(offs_site, par_site, n_dad) %>%
        group_by(offs_site, par_site) %>%
        summarise(n_dad=(sum(n_dad))) }
    else {site_matM <- as.data.frame(matrix(nrow=1, ncol=3))
          site_matM[1,1] <- site_eval
          site_matM[1,2] <- site_eval
          site_matM[1,3] <- 0
          col <- c("offs_site", "par_site", "n_dad")
        colnames(site_matM) <- col
         }
    site_matM2 <- suppressWarnings(left_join(sites, site_matM, by=c(site ="par_site"))) 
        site_matM3 <- site_matM2 %>%
        select(n_dad)
    
#now bind_cols
       #order matters here--> mums need to be in the righthand column, join so sites without matches appear as zero values rather than absent
   site_mat_fin <- bind_cols(site_matM3, site_matF3)
        
    #need to transpose so that it's formatted for migest
    site_mat_T <- as.data.frame(t(site_mat_fin))
    site_mat_T[is.na(site_mat_T)] <- 0
    
    #generate header
    site_head <- as.data.frame(matrix(nrow=2, ncol=2))
    col <- c("male", "female")
    colnames(site_head) <- col
    
    #proportion sampled
    if(site_eval %in% N_cap2$site){
    site_head_prop <- prop_samp %>% 
    filter(site== site_eval) %>%
    rename(n_or_p="prop_habitat_sampled") }
    else {site_head_prop <- as.data.frame(matrix(nrow=1, ncol=1))
          site_head_prop[1,1] <- 0
          col <- "n_or_p"
        colnames(site_head_prop) <- col
        
    }
    
    
    #number of captured adults
    if(site_eval %in% N_cap2$site){
        site_head_ncap <- N_cap2 %>%
        filter(site== site_eval) %>%
        select(n_male, n_female) %>%
        rename(n_or_p_M="n_male", n_or_p_F="n_female")} 
    else {site_head_ncap <- as.data.frame(matrix(nrow=1, ncol=2))
          site_head_ncap[1,1] <- 0
          site_head_ncap[1,2] <- 0
          col <- c("n_or_p_M", "n_or_p_F")
        colnames(site_head_ncap) <- col
         }
    
    #make a male/female column of proportion sampled
    site_head_prop2 <- site_head_prop %>%
    select(n_or_p) %>%
    mutate(n_or_p_F=n_or_p) %>%
    rename(n_or_p_M=n_or_p)
    
    #bind header rows
    site_head <- bind_rows(site_head_prop2, site_head_ncap)
    
    #add 16 extra columns so the table writes properly
    empt_dat1 <- as.data.frame(matrix(nrow=2, ncol=(nrow(sites)-2)))
    site_head2 <- bind_cols(site_head, empt_dat1)
    
    #rename columns consistently for binding
    col <- c(sites$pop)
    colnames(site_head2) <- col
    colnames(site_mat_T) <- col
    
    #add the parentage matrix
    site_with_parmat <- bind_rows(site_head2, site_mat_T)
    
    #add trio counts
    site_trios <- N_trios %>%
    filter(offs_site==site_eval)
    #filter(year %in% year)


    
    site_trios_with_sites <- suppressWarnings(left_join(sites, site_trios, by=c(site="par_site")))
    site_trios_with_sites <- site_trios_with_sites %>% select(n_trios)
    site_trios_with_sites_T <- as.data.frame(t(site_trios_with_sites))
    site_trios_with_sites_T[is.na(site_trios_with_sites_T)] <- 0
    
    #bind trios to parentage matrix
    colnames(site_trios_with_sites_T) <- col
    site_trios_with_sites_T[is.na(site_trios_with_sites_T)] <- 0
    site_with_trios <- bind_rows(site_with_parmat, site_trios_with_sites_T)
    
    #add the number of offspring sampled
    if(site_eval %in% N_cap_offs2$site){
    site_offs <- N_cap_offs2 %>%
    filter(site== site_eval) %>%
    select(sampled_fish)}
    else {site_offs <- as.data.frame(matrix(nrow=1, ncol=1))
          site_offs[1,1] <- 0
           }
    
    #build data frame for offspring sampled count
    offs_dat <- as.data.frame(matrix(nrow=1, ncol=nrow(sites)))
    offs_dat[1,1] <- site_offs
    colnames(offs_dat) <- col
    
    #pull together whole data table
    site_with_offs <- bind_rows(site_with_trios, offs_dat) #site_with_offs
    
    #add space line so populations are separeated in the input table
    space <- as.data.frame(matrix(nrow=1, ncol=(nrow(sites))))
    colnames(space) <- col
    site_fin <- bind_rows(site_with_offs, space)
    colnames(site_fin) <- col
    
    #bind together complete tables for each site
    if(rowSums(site_fin[2,], na.rm=TRUE)>0) {empt_dat2 <- bind_rows(empt_dat2, site_fin)}
    
    #check sums of parentage matches
    sum_par[i,] <-(sum(site_mat_T, na.rm=TRUE))
    sum_trios[i,] <-(sum(site_trios_with_sites_T, na.rm=TRUE))
    
    #output sites included
    site_eval <- as.data.frame(site_eval)
    col <- "site"
    colnames(site_eval) <- col
    site_eval$site <- as.character(site_eval$site)
    if(rowSums(site_fin[2,], na.rm=TRUE)>0) {site_inc <- bind_rows(site_inc, site_eval)}

}
       

In [10]:
dim(empt_dat2) #should be npop*7 X npop

[1] 133  20

In [11]:
sum(dads$n_dad, na.rm= TRUE)
sum(mums$n_mum, na.rm=TRUE)
#should add to 76, plus 14 trios and we have 90

[1] 18

[1] 20

In [12]:
sum(sum_par, na.rm=TRUE) #should match the number of singular parentage matches

[1] 38

In [13]:

sum(sum_trios, na.rm=TRUE) #should match the number of trios

[1] 24

In [14]:
24+38

[1] 62

In [15]:
n_pop <- nrow(empt_dat2)/7
n_pop

[1] 19

In [16]:
#write the site lists for use in processing output
sites_fin <- site_inc %>%
    select(site) 

sites_fin$pop <- seq(from=1, to=n_pop, by=1)
dim(sites_fin)
sites_fin$site <- gsub(" ", "_", sites_fin$site, fixed=TRUE)


[1] 19  2

In [17]:
#stupid janky fix
empt_dat2 <- empt_dat2[,1:19]

In [18]:
#prepare the file table header
header <- as.data.frame(matrix(nrow=4, ncol=(nrow(site_inc))))
col <- c(sites_fin$pop)
colnames(header) <- col
header[1,1] <- (nrow(empt_dat2)/7) #7 rows of input for 1 population 
if((nrow(empt_dat2)/7) < 20){header[2,1] <- 0} else {header[2,1] <- 1} #1 yes all populations sampled, 0 no
header[3,1] <- 0.0 #no prior weight for now
header[4,1] <- "Migest_allyears.out"
header_fin <- bind_rows(header, space)

In [19]:
migest_in <- rbind(header_fin, empt_dat2)


In [275]:
#migest_in

In [20]:
## write the site lists for use in processing output
write.table(sites_fin, file="~/migest/20190612_1340loci/all_years/input_sites_allyears.txt", row.names = FALSE, col.names = TRUE, quote=FALSE)

In [79]:
write.table(migest_in, file="~/migest/20190612_1340loci/all_years/MigEst.in", quote=FALSE, na=" ", sep=" ", col.names=FALSE, row.names=FALSE,)